In [15]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
import numpy as np
import pandas as pd
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchmetrics


In [16]:
def create_sequences(df, seq_length):
    xs, ys = [], []
    # Iterate over data indices
    for i in range(len(df) - seq_length):
      	# Define inputs
        x = df.iloc[i:(i+seq_length), 1]
        # Define target
        y = df.iloc[i+seq_length, 1]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

In [17]:
train_data = pd.read_csv("electricity_consump/electricity_train.csv")

# Use create_sequences to create inputs and targets
X_train, y_train = create_sequences(train_data, 24*4)
print(X_train.shape, y_train.shape)

# Create TensorDataset
dataset_train = TensorDataset(
    torch.from_numpy(X_train).float(),
    torch.from_numpy(y_train).float(),
)
print(len(dataset_train))

dataloader_train = DataLoader(
  dataset_train, shuffle=True, batch_size=16
)

(105119, 96) (105119,)
105119


In [18]:
test_data = pd.read_csv("electricity_consump/electricity_test.csv")

# Use create_sequences to create inputs and targets
X_test, y_test = create_sequences(test_data, 24*4)
print(X_test.shape, y_test.shape)

# Create TensorDataset
dataset_test = TensorDataset(
    torch.from_numpy(X_test).float(),
    torch.from_numpy(y_test).float(),
)
print(len(dataset_test))

dataloader_test = DataLoader(
  dataset_test, shuffle=True, batch_size=16
)

(34944, 96) (34944,)
34944


In [19]:
# class Net(nn.Module):
#     def __init__(self):
#         super().__init__()
#         # Define RNN layer
#         self.rnn = nn.RNN(
#             input_size=1,
#             hidden_size=32,
#             num_layers=2,
#             batch_first=True,
#         )
#         self.fc = nn.Linear(32, 1)
# 
#     def forward(self, x):
#         # Initialize first hidden state with zeros
#         h0 = torch.zeros(2, x.size(0), 32)
#         # Pass x and h0 through recurrent layer
#         out, _ = self.rnn(x, h0)  
#         # Pass recurrent layer's last output through linear layer
#         out = self.fc(out[:, -1, :])
#         return out

In [20]:
class Net(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        # Define lstm layer
        self.lstm = nn.LSTM(
            input_size=1,
            hidden_size=32,
            num_layers=2,
            batch_first=True,
        )
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        h0 = torch.zeros(2, x.size(0), 32)
        # Initialize long-term memory
        c0 = torch.zeros(2, x.size(0), 32)
        # Pass all inputs to lstm layer
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [21]:
# class Net(nn.Module):
#     def __init__(self):
#         super().__init__()
#         # Define RNN layer
#         self.gru = nn.GRU(
#             input_size=1,
#             hidden_size=32,
#             num_layers=2,
#             batch_first=True,
#         )
#         self.fc = nn.Linear(32, 1)
# 
#     def forward(self, x):
#         h0 = torch.zeros(2, x.size(0), 32)
#         out, _ = self.gru(x, h0)  
#         out = self.fc(out[:, -1, :])
#         return out

In [22]:
net = Net(input_size=1)
# Set up MSE loss
criterion = nn.MSELoss()
optimizer = optim.Adam(
  net.parameters(), lr=0.0001
)

for epoch in range(3):
    for seqs, labels in dataloader_train:
        # Reshape model inputs
        seqs = seqs.view(16, 96, 1)
        # Get model outputs
        outputs = net(seqs)
        # Compute loss
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

/Users/kawsar/anaconda3/envs/python_3_10/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: shape '[16, 96, 1]' is invalid for input of size 1440

In [ ]:
# Define MSE metric
mse = torchmetrics.MeanSquaredError()

net.eval()
with torch.no_grad():
    for seqs, labels in dataloader_test:
        seqs = seqs.view(32, 96, 1)
        # Pass seqs to net and squeeze the result
        outputs = net(seqs).squeeze()
        mse(outputs, labels)

# Compute final metric value
test_mse = mse.compute()
print(f"Test MSE: {test_mse}")